In [ ]:
import numpy as np
import pandas as pd

df_learning = pd.read_csv("../data-source/202208-202301.csv")
# df_learning = pd.read_csv("../data-source/201807-202207.csv")

In [ ]:
import jieba

def cutReview(x): return ' '.join(jieba.lcut(str(x), cut_all=False))

df_learning.loc[:, '类型'] = df_learning['类型'].fillna(
    '')
# df_learning.loc[:, '交易对方'] = df_learning['交易对方'].fillna(
#     '').apply(cutReview)
# df_learning.loc[:, '商品名称'] = df_learning['商品名称'].fillna(
#     '').apply(cutReview)

df_learning.loc[:, '交易对方'] = df_learning['交易对方'].fillna(
    '')
df_learning.loc[:, '商品名称'] = df_learning['商品名称'].fillna(
    '')


# 模型训练

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import PCA
from scipy.sparse import hstack

# 去除词频小于2的词
countvec = CountVectorizer(min_df=2)

# 构建特征向量
product_vector = countvec.fit_transform(df_learning['商品名称'])
saler_vector = countvec.fit_transform(df_learning['交易对方'])

# 合并特征向量
feature_vector = hstack((product_vector, saler_vector))

# 特征降维
pca = PCA(n_components=50)
feature_vector_pca = pca.fit_transform(feature_vector.toarray())

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# 按照 7:3 划分训练集和测试集
x_train, x_test, y_train, y_test = train_test_split(
    feature_vector_pca, df_learning['类型'], test_size=0.3)

svm_model = SVC(kernel='rbf', probability=True, random_state=1)
svm_model.fit(x_train, y_train)

y_predict = svm_model.predict(x_test)

# 计算准确度
acc = accuracy_score(y_test, y_predict)
print("训练模型准确率为：{:.2f}%".format(acc * 100))


# 预测集填入

In [ ]:
df_new = pd.read_csv("../out/merged.csv")

df_predict = df_new.copy()

def cutReview(x): return ' '.join(jieba.lcut(str(x), cut_all=False))

# df_predict.loc[:, '交易对方'] = df_predict['交易对方'].fillna(
#     '').apply(cutReview)
# df_predict.loc[:, '商品名称'] = df_predict['商品名称'].fillna(
#     '').apply(cutReview)

df_predict

In [ ]:
# 去除词频小于2的词
countvec = CountVectorizer(min_df=2)

# 构建特征向量
product_vector = countvec.fit_transform(df_predict['商品名称'])
saler_vector = countvec.fit_transform(df_predict['交易对方'])

# 合并特征向量
feature_vector = hstack((product_vector, saler_vector))

# 特征降维
pca = PCA(n_components=50)
feature_vector_pca = pca.fit_transform(feature_vector.toarray())

# 结果预测

In [ ]:
confidence_threshold = 0.6

y_predict = svm_model.predict(feature_vector_pca)
confidence_scores = svm_model.predict_proba(feature_vector_pca)

# 根据阈值过滤不可信的结果
for i in range(len(y_predict)):
    if np.max(confidence_scores[i]) < confidence_threshold:
        y_predict[i] = ''

df_new['类型'] = y_predict

df_new


In [ ]:
df_new.to_csv("../out/predict_result.csv", index=False, header=True, encoding='utf-8')